In [1]:
import json
import numpy as np
import pandas as pd
from datetime import timedelta
from geopy.distance import distance, geodesic
from geopy import Point, distance as geo_distance
from geographiclib.geodesic import Geodesic as GD
from scipy.interpolate import interpn
import folium
from datetime import datetime, timedelta
import pytz
import torch
import torch.nn as nn

np.set_printoptions(precision=4, suppress=True)

## steps
1. read in files (json or csv) and parse the data into dataframe
2. check if timestamp is in accending order
3. longitude convertion: western negative to 0-360 format
4. according to the current position and timestamp, filter out the needed waypoints to df_filter
5. create section_waypoints
6. generate interpolated coordinates, timestamps and write cog
7. interpolate corresponding weather attributes for all the spatial points in the experimenting area
8. do the transformation for weather attributes (difference between previous forecasted and latest forecasted)
9. zero padding
10. pass through the model

In [2]:
# longitude list and latitude list of the grids in the exaperimenting area
long_list = np.array([156.00000694, 157.25000699, 158.50000705, 159.7500071 ,
                      161.00000716, 162.25000722, 163.50000727, 164.75000733,
                      166.00000738, 167.25000744, 168.50000749, 169.75000755,
                      171.00000761, 172.25000766, 173.50000772, 174.75000777,
                      176.00000783, 177.25000788, 178.50000794, 179.75000799,
                      181.00000805, 182.25000811, 183.50000816, 184.75000822,
                      186.00000827, 187.25000833, 188.50000838, 189.75000844,
                      191.00000849, 192.25000855, 193.50000861, 194.75000866,
                      196.00000872, 197.25000877, 198.50000883, 199.75000888,
                      201.00000894, 202.250009  , 203.50000905, 204.75000911,
                      206.00000916, 207.25000922, 208.50000927, 209.75000933,
                      211.00000938, 212.25000944, 213.5000095 , 214.75000955,
                      216.00000961, 217.25000966, 218.50000972, 219.75000977,
                      221.00000983, 222.25000988, 223.50000994, 224.75001   ,
                      226.00001005, 227.25001011, 228.50001016])
lat_list = np.array([50.  , 49.75, 49.5 , 49.25, 49.  , 48.75, 48.5 , 48.25, 48.  ,
       47.75, 47.5 , 47.25, 47.  , 46.75, 46.5 , 46.25, 46.  , 45.75,
       45.5 , 45.25, 45.  , 44.75, 44.5 , 44.25, 44.  , 43.75, 43.5 ,
       43.25, 43.  , 42.75, 42.5 , 42.25, 42.  , 41.75, 41.5 , 41.25,
       41.  , 40.75, 40.5 , 40.25, 40.  , 39.75, 39.5 , 39.25, 39.  ,
       38.75, 38.5 , 38.25, 38.  , 37.75, 37.5 , 37.25, 37.  , 36.75,
       36.5 , 36.25, 36.  , 35.75, 35.5 , 35.25, 35.  , 34.75, 34.5 ,
       34.25, 34.  , 33.75, 33.5 , 33.25, 33.  , 32.75, 32.5 , 32.25,
       32.  , 31.75, 31.5 , 31.25, 31.  , 30.75, 30.5 , 30.25, 30.  ,
       29.75, 29.5 , 29.25, 29.  , 28.75, 28.5 , 28.25, 28.  , 27.75,
       27.5 , 27.25, 27.  , 26.75, 26.5 , 26.25, 26.  , 25.75, 25.5 ,
       25.25])

range_ul = [50, 156.00000694]  # upper left corner coordinates of the experimenting area
range_lr = [25, 228.50001016]  # lower right corner coordinates of the experimenting area

### 1. load input data

In [3]:
# Option 1: load the JSON file
# the provided example is not in the northen pacific area
with open('input_sample.json', 'r') as f:
    data = json.load(f)

# create a dictionary to store the data
parsed_data = {'latitude': [], 'longitude': [], 'timestamp': []}

# iterate over each coordinate and timestamp, and store them in the dictionary
for i in range(len(data['coordinates'])):
    parsed_data['latitude'].append(data['coordinates'][i]['latitude'])
    parsed_data['longitude'].append(data['coordinates'][i]['longitude'])
    parsed_data['timestamp'].append(data['timestamps'][i])

# create a Pandas DataFrame from the parsed data
input_df = pd.DataFrame(parsed_data)


In [4]:
# Option 2: read the csv file into a DataFrame
input_df = pd.read_csv('98765434_voyage-export.csv')

# extract the desired variables
variables = ['LATITUDE', 'LONGITUDE', 'TIME AT WAYPOINT']
input_df = input_df[variables]
input_df = input_df.rename(columns={'TIME AT WAYPOINT': 'timestamp', 'LATITUDE': 'latitude', 'LONGITUDE': 'longitude'})


### 2 + 3 timestamp and longitude conversion 

In [5]:
# convert longitudes in the western hemisphere to positive numbers
input_df.loc[input_df['longitude'] < 0, 'longitude'] += 360
# convert timestamp
input_df['timestamp'] = pd.to_datetime(input_df['timestamp'], format='%Y-%m-%d %H:%M:%S')
if not input_df['timestamp'].is_monotonic_increasing:
    # Sort the DataFrame by the timestamp column
    input_df = input_df.sort_values('timestamp')

### 4. filter needed waypoints according to current T0 and current position G0 (lat0, lon0) 

In [6]:
#current T0 and current position G0 (lat0, lon0) should be provided by json file
# Now define T0 and G0 as examples

# example 1: starting point is where the ship not yet entered the experimenting area
T0_str = '2022-10-09 17:05:13+00:00'
G0 = (32,124)
T0 = pd.Timestamp(T0_str, tz='UTC')

# example 2: starting point is where the ship is in the middle of the experimenting area
#T0_str = '2022-10-14 16:28:15+00:00'
#G0 = (49,171)
#T0 = pd.Timestamp(T0_str, tz='UTC')

# example 3: starting point is where the ship has already sailed outside the experimenting area
#T0_str = '2022-10-21 07:07:33+00:00'
#G0 = (34.313492,239.532944)
#T0 = pd.Timestamp(T0_str, tz='UTC')
           
# filter the dataframe by timestamp and longitude
mask = (input_df['timestamp'] >= T0) & (input_df['longitude'].between(range_ul[1], range_lr[1]))

# check if the current point or timestamp is after/ exceed the experimenting area
if mask.any():
    # do the filtering
    index_first = np.nonzero(mask.values)[0][0]
    index_last = np.nonzero(mask.values)[0][-1]
    df_filtered = input_df[index_first-1:index_last+2]
else:
    # Report
    print('Error: rerouting point outside the right bound of the experimenting area')
    # also need to stop excuting at all

'''in case of the 3nd senario, quit from here'''

### 5. waypoints_section

In [7]:
# store the information between each pair of consecutive waypoints from df_filtered in sections_waypoints

# calculating bearing
def get_bearing(lat1, lat2, long1, long2):
    '''The function is used to calculate the CoG from node 1 to node 2'''
    brng = GD.WGS84.Inverse(lat1, long1, lat2, long2)['azi1']
    if brng < 0:
        brng = brng + 360
    return brng


sections_waypoints = pd.DataFrame(columns=['E_latitude', 'E_longitude', 'L_latitude', 
                                                'L_longitude', 'E_timestamp', 'L_timestamp', 'cog'])

for i in range(len(df_filtered) - 1):
    row1 = df_filtered.iloc[i]
    row2 = df_filtered.iloc[i + 1]

    # Calculate course over ground between two waypoints
    cog = get_bearing(row1['latitude'], row2['latitude'], row1['longitude'], row2['longitude'])

    # Add the information to the sections_waypoints dataframe
    sections_waypoints = sections_waypoints.append({
            'E_latitude': row1['latitude'],
            'E_longitude': row1['longitude'],
            'E_timestamp': row1['timestamp'],
            'L_latitude': row2['latitude'],
            'L_longitude': row2['longitude'],
            'L_timestamp': row2['timestamp'],
            'cog': cog
    }, ignore_index=True)
sections_waypoints['E_latitude'] = sections_waypoints['E_latitude'].astype(float)
sections_waypoints['E_longitude'] = sections_waypoints['E_longitude'].astype(float)
sections_waypoints['L_latitude'] = sections_waypoints['L_latitude'].astype(float)
sections_waypoints['L_longitude'] = sections_waypoints['L_longitude'].astype(float)
sections_waypoints['E_timestamp'] = pd.to_datetime(sections_waypoints['E_timestamp'])
sections_waypoints['L_timestamp'] = pd.to_datetime(sections_waypoints['L_timestamp'])
sections_waypoints['cog'] = sections_waypoints['cog'].astype(float)

/var/folders/fp/9nc54f7n0z566skqx52tb5340000gp/T/ipykernel_48362/37114126.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sections_waypoints = sections_waypoints.append({
/var/folders/fp/9nc54f7n0z566skqx52tb5340000gp/T/ipykernel_48362/37114126.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sections_waypoints = sections_waypoints.append({
/var/folders/fp/9nc54f7n0z566skqx52tb5340000gp/T/ipykernel_48362/37114126.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sections_waypoints = sections_waypoints.append({
/var/folders/fp/9nc54f7n0z566skqx52tb5340000gp/T/ipykernel_48362/37114126.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat 

### 6. generate interpolated coordinates, timestamp and write cog

In [8]:
# Create an empty pandas DataFrame with column 'longitude'
interpolation_full = pd.DataFrame({'longitude': long_list})

# Add empty columns to the DataFrame
interpolation_full['latitude'] = np.nan    # longitudes are given according to the grids
                                           # latitudes are interpolated
interpolation_full['cog'] = np.nan         
interpolation_full['timestamp'] = pd.NaT
interpolation_full['arriving_time_latest'] = np.nan
interpolation_full['arriving_time_previous'] = np.nan

In [9]:
def intp_lat_ts_cog(row):
    """
    Finds the row in `sections_waypoints` where the logitudes in the grid `t_lon` falls between
    the `E_longitude` and `L_longitude` values. Calculates the latitude
    and timestamp for the target point T_point and sets the `latitude`,
    `timestamp`, and `cog` columns in accordingly.

    Parameters:
    t_lon (float): The longitude value to search for in `sections_waypoints`.
    sections_waypoints (pd.DataFrame): The DataFrame containing the sections and waypoints.

    Returns:
    pd.DataFrame: a row with interpolated values of latitude, timestamp and cog
    """


    # Get t_lon value 
    t_lon = row['longitude']

    # Find row in `sections_waypoints` where `t_lon` falls between
    #         the `E_longitude` and `L_longitude` values
    mask = (sections_waypoints['E_longitude'] <= t_lon) & (t_lon <= sections_waypoints['L_longitude'])
    row_sw = sections_waypoints.loc[mask]

    # Calculate t_lat using linear interpolation
    point1 = np.array([row_sw['E_latitude'], row_sw['E_longitude']])
    point2 = np.array([row_sw['L_latitude'], row_sw['L_longitude']])
    t_lat = point1[0] + ((t_lon - point1[1]) / (point2[1] - point1[1])) * (point2[0] - point1[0])

    # Calculate t_timestamp using linear interpolation
    e_ts = row_sw['E_timestamp']
    l_ts = row_sw['L_timestamp']

    t_ts = e_ts + ((t_lon - point1[1])[0] / (point2[1] - point1[1])[0]) * (l_ts - e_ts)

    return pd.Series({'latitude': t_lat[0], 'timestamp': t_ts.values[0], 'cog': row_sw['cog'].iloc[0]})

In [10]:
# return the index of the smallest longitude in grids that is larger than the current position G0[1]
s_grids_lon_index = (interpolation_full['longitude'] >= G0[1]).idxmax()

In [11]:
#interpolate latitude timestamp and write cog
interpolation_full.loc[s_grids_lon_index:, ['latitude', 'timestamp', 'cog']] = (interpolation_full.loc[s_grids_lon_index:,:]
                                                                                .apply(intp_lat_ts_cog, axis=1))

### 7. read and interpolate corresponding weather attributes

In [12]:
# find out the lastest available weather forecast according to the current timepoint

def get_last_weather_forecast(timestamp_str):
    
    '''return the forecast files names and the timestamps of the forecast'''
    # only keep the year month day hour minute seconds
    truncated_timestamp_str = timestamp_str[:19]
    
    dt = datetime.strptime(truncated_timestamp_str, '%Y-%m-%d %H:%M:%S')
    forecast_hour = (dt.hour // 6) * 6
    last_forecast = pytz.utc.localize(datetime(dt.year, dt.month, dt.day, forecast_hour))
    previous_forecast = last_forecast-timedelta(hours = 6)
    
    # the most recent forecast and the previous one
    last_forecast_name = (str(last_forecast.year)+str(last_forecast.month).zfill(2)+str(last_forecast.day).zfill(2)
                             +str(last_forecast.hour).zfill(2))
    
    previous_forecast_name = (str(previous_forecast.year)+str(previous_forecast.month).zfill(2)
                              +str(previous_forecast.day).zfill(2)+str(previous_forecast.hour).zfill(2))
    return (last_forecast_name, previous_forecast_name),(last_forecast, previous_forecast) 


In [13]:
forecast_name, forecast_ts = get_last_weather_forecast(T0_str)

In [14]:
# Define a function to calculate the time difference in hours 
# between the arriving time at a location and the starting time of the weather forecast
time_diff0 = lambda x: (pytz.utc.localize(x['timestamp']) - forecast_ts[0]) / timedelta(hours=1)
time_diff1 = lambda x: (pytz.utc.localize(x['timestamp']) - forecast_ts[1]) / timedelta(hours=1)

# Apply the function to the dataframe
interpolation_full.loc[s_grids_lon_index:, 'arriving_time_latest'] = (interpolation_full.loc[s_grids_lon_index:,:]
                                                                      .apply(time_diff0, axis=1))
interpolation_full.loc[s_grids_lon_index:, 'arriving_time_previous'] = (interpolation_full.loc[s_grids_lon_index:,:]
                                                                      .apply(time_diff1, axis=1))

In [16]:
def hour2timeframe(row):
    '''The timeframe is from 0 to 209, but the real hour is from 0 to 384. 
    From hour 120 onwards, every three hours per timeframe.
    The function is to find out the positions in the timeframes according to the hour'''
    if row['arriving_time_latest'] > 120:
        tf_latest = (row['arriving_time_latest']-120)/3 + 120
    else:
        tf_latest = row['arriving_time_latest']
        
    if row['arriving_time_previous'] > 120:
        tf_previous = (row['arriving_time_previous']-120)/3 + 120
    else:
        tf_previous = row['arriving_time_previous']
    return pd.Series({'tf_latest': tf_latest, 'tf_previous': tf_previous})

In [17]:
interpolation_full.loc[s_grids_lon_index:, ['tf_latest', 'tf_previous']] = (interpolation_full.loc[s_grids_lon_index:,:]
                                                                      .apply(hour2timeframe, axis=1))

In [19]:
# read in the needed weather forecast file
wf_latest = np.load('/gfs_NP_'+forecast_name[0]+'.npy')
wf_previous = np.load('/gfs_NP_'+forecast_name[1]+'.npy')

In [20]:
#flip for the 1st dimension (latitude) because interpolation requires first dimension in accending order
# slice and drop the last longitude as weather attributes are not needed for the destination
wf_latest = np.flip(wf_latest, axis = 0)[:,:-1,:,:]
wf_previous = np.flip(wf_previous, axis = 0)[:,:-1,:,:]

In [21]:
def interp_row(row, wf, tsn):
    
    '''
    The function aims to retrieve and interpolate the weather conditions at the given position according 
    to its spatial coordinates and arriving time (hour)
    input:
    wf is the weather forecast array, wf_latest or wf_previous
    tsn = 'tf_latest' or 'tf_previous' according to wf
    output: weather conditions for wind, wave and swell (8)
    '''
    # Define the interpolation points as a tuple of arrays
    interp_points = (row['latitude'], row['longitude'], row[tsn], range(wf.shape[3]))

    # Interpolate the weather variable values at the interpolation points
    var_values = interpn((np.flip(lat_list), long_list, range(wf.shape[2]), range(wf.shape[3])),
                     wf, interp_points, method='linear')
    return var_values

In [22]:
# interpolation of weather conditions forecasted in the latest weather forecast and the previous weather forecast
interpolation_full.loc[s_grids_lon_index:,'latest_weather'] = interpolation_full.loc[s_grids_lon_index:,:].apply(lambda row: 
                                                                                                                 interp_row(row, wf_latest, 'tf_latest'), axis = 1)
interpolation_full.loc[s_grids_lon_index:,'previous_weather'] = interpolation_full.loc[s_grids_lon_index:,:].apply(lambda row: 
                                                                                                                   interp_row(row, wf_previous, 'tf_previous'), axis = 1)

In [24]:
# Define a function that converts a 1D numpy array of weather variables to the desired format
def convert_weather_array(weather_array, cog):
    # Make a copy of the input array of weather conditions
    converted_array = np.copy(weather_array)

    # Define the indices of the directions of wind, wave and swell
    direction_indices = [1, 4, 7]

    # Subtract the cog from the direction and take the absolute value and convert if larger than 180 degree
    for idx in direction_indices:
        diff = abs(weather_array[idx] - cog)
        if diff <= 180:
            converted_array[idx] = diff
        else:
            converted_array[idx] = 360 - diff

    return converted_array


# Apply the 'convert_weather_array' function to each row of the DataFrame
converted_weather = interpolation_full.loc[s_grids_lon_index:,:].apply(lambda row: convert_weather_array(
    row['latest_weather'], row['cog']) - convert_weather_array(row['previous_weather'], row['cog']), axis=1)

# Concatenate the converted weather arrays into a single flattened array
flattened_weather = np.concatenate(converted_weather.to_numpy())

# Zero-pad the flattened array if necessary
if len(flattened_weather) < 59 * 8:
    padding = np.zeros((59 * 8 - len(flattened_weather),))
    flattened_weather = np.concatenate((padding, flattened_weather))



### inference

In [26]:
# architecture:
class BinaryClassification(nn.Module):
    def __init__(self):
        super(BinaryClassification, self).__init__()
        # Number of input features is 12.
        self.layer_1 = nn.Linear(472, 640) 
        self.layer_2 = nn.Linear(640, 120)
        self.layer_3 = nn.Linear(120, 64)
        self.layer_out = nn.Linear(64, 1) 
        
        self.relu = nn.ReLU()
        self.dropout1 = nn.Dropout(p=0.1)
        self.dropout2 = nn.Dropout(p=0.2)
        self.batchnorm1 = nn.BatchNorm1d(640)
        self.batchnorm2 = nn.BatchNorm1d(120)
        self.batchnorm3 = nn.BatchNorm1d(64)
    def init_weights(self):
        torch.nn.init.kaiming_normal_(self.layer_1.weight)
        torch.nn.init.kaiming_normal_(self.layer_2.weight)
        torch.nn.init.kaiming_normal_(self.layer_3.weight)
        torch.nn.init.kaiming_normal_(self.layer_out.weight)      
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout1(x)
        x = self.relu(self.layer_3(x))
        x = self.batchnorm3(x)
        x = self.dropout2(x)
        x = self.layer_out(x)
        
        return x
model = BinaryClassification()

In [27]:
# load the trained model
trained_model = torch.load('pretrained_wr_v2.pt')['state_dict']
model.load_state_dict(trained_model)

<All keys matched successfully>

In [28]:
# predict
tensor_input = torch.tensor(flattened_weather).float()
tensor_input = tensor_input.reshape(1, -1)
model.eval()
probs = torch.sigmoid(model.forward(tensor_input))
preds = (probs >= 0.5).int()
print('Rerouting is recommended!' if preds == 1 else 'Rerouting is not recommended!')

Rerouting is not recommended!
